In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file = "../path_to_data/data_info.xlsx"

df = pd.read_excel(file)

In [ ]:
attributes = [
    "Mapping ID",
    "DIAGNOSIS",
    "label",
    "AGE",
    "SEX",
    "TYPE CTD",
    "TYPE CTD OTHER",
    "SMOKING HISTORY",
    "CTD DIAG DATE",
    "PACK-YEARS",
    "SMOKING HISTORY STOP DATE",
    "RISK OCCUPATIONAL", 
    "RISK ENVIRONMENTAL", 
    "RISK PRECIPITINS", 
    "FAMILIAL PF AND/OR RARE VARIANT", 
]

In [ ]:
subset_df = df[attributes]

In [ ]:
subset_df["label"] = subset_df["label"].replace({1: "Alive", 0: "Dead"})
subset_df.rename(
    columns={"label": "State of (Dead/Alive) within one year"}, inplace=True
)

In [ ]:
subset_df["PACK-YEARS"] = subset_df["PACK-YEARS"].replace(
    {99999.0: np.nan, 0.0: np.nan}
)

In [ ]:
subset_df["SMOKING HISTORY STOP DATE"] = subset_df["SMOKING HISTORY STOP DATE"].replace(
    {99999.0: np.nan, 0.0: np.nan}
)
# Correcting negative values in 'SMOKING HISTORY STOP DATE' column
subset_df.loc[
    subset_df["SMOKING HISTORY STOP DATE"] < 0, "SMOKING HISTORY STOP DATE"
] = subset_df.loc[
    subset_df["SMOKING HISTORY STOP DATE"] < 0, "SMOKING HISTORY STOP DATE"
].abs()

In [ ]:
subset_df["TYPE CTD"] = subset_df["TYPE CTD"].replace({"Not Available": np.nan})

In [ ]:
prompt_results = {}

special_dealing = {
    "SMOKING HISTORY STOP DATE": "Stopped smoking {} days ago.",
    "AGE": "Patient age: {}.",
    "SEX": lambda x: (
        "Gender: Male."
        if x == "M"
        else ("Gender: Female." if x == "F" else "Gender: Unspecified.")
    ),
    "TYPE CTD": "CTD diagnosis: {}.",
    "PACK-YEARS": "{}",
    "RISK OCCUPATIONAL": lambda x: (
        "Occupational exposure present." if x == "Yes" else "No occupational exposure."
    ),
    "RISK ENVIRONMENTAL": lambda x: (
        "Environmental risk factors present." if x == "Yes" else " "
    ),
    "CTD DIAG DATE": lambda x: f"CTD diagnosed {abs(x)} days ago.",
}


# Assuming 'subset_df' is your DataFrame
for index in range(len(subset_df)):
    row = subset_df.iloc[index]

    # Generate the prompt, skipping attributes with NaN values
    prompt_parts = []
    for attr in subset_df.columns:
        if attr != "Mapping ID" and not pd.isna(row[attr]):
            if attr in special_dealing:
                special_format = special_dealing[attr]
                # Check if the special handling is a function and call it
                if callable(special_format):
                    prompt_parts.append(special_format(row[attr]))
                else:
                    prompt_parts.append(special_format.format(row[attr]))
            else:
                # Standard format for other attributes
                prompt_parts.append(f"{attr} is {row[attr]},")

    prompt = "Lung CT scan : " + " ".join(prompt_parts).rstrip(",") + "."

    # Store the generated prompt using 'Mapping ID' as the key
    prompt_results[row["Mapping ID"]] = prompt
    print(prompt)

In [ ]:
print("prompt_results: ", prompt_results)
prompt_results_list = [
    (file_name, prompt) for file_name, prompt in prompt_results.items()
]

In [ ]:
prompt_results_df = pd.DataFrame(prompt_results_list, columns=["File Name", "Prompt"])

In [ ]:
one = list(
    prompt_results_df[prompt_results_df["File Name"] == "AIIB23_4.nii.gz"].Prompt
)
print(one)

In [ ]:
prompt_results_df.to_csv(
    "../path_to_data/data_info_all_prompts.csv"
)